In [ ]:
!sudo apt-get install unrar
!pip install patool

In [ ]:
import os
import re
import json
import patoolib
import pandas as pd

In [ ]:
# Путь к RAR-архиву
rar_file = '/content/all_annotated.rar'

# Папочка для распаковки
extract_folder = 'my_data_2'
os.makedirs(extract_folder, exist_ok=True)

patoolib.extract_archive(rar_file, outdir=extract_folder)

In [ ]:
# Папка с JSON-файлами
folder_path = '/content/my_data_2/all_annotated'

# Списки для хранения текстов с лейблами и без
texts_with_euphemism = []
texts_without_euphemism = []

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)

        # Чтение JSON-файла
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)

            for item in data:
                # Извлекаем текст
                text = item.get('text', '')

                # Проверка наличия лейбла <euphemism>
                if '<euphemism>' in text:
                    texts_with_euphemism.append({
                        'filename': filename,
                        'index': item.get('index', None),  # Сохраняем индекс
                        'text': text
                    })
                else:
                    texts_without_euphemism.append({
                        'filename': filename,
                        'index': item.get('index', None),
                        'text': text
                    })

# Создание датафреймов
df_with_euphemism = pd.DataFrame(texts_with_euphemism)
df_without_euphemism = pd.DataFrame(texts_without_euphemism)

In [ ]:
print("Тексты с лейблом <euphemism>:")
print(df_with_euphemism)

In [ ]:
df_with_euphemism.to_csv('only_euphemism_dataset.csv')

In [ ]:
print("\nТексты без лейбла <euphemism>:")
print(df_without_euphemism)

In [ ]:
df_without_euphemism.dropna()

In [ ]:
len(df_without_euphemism)

In [ ]:
# Следующие ячейки используются после ручной проверки текстов с эвфемизмами

In [ ]:
# Загружаем проверенный датасет с эвфемизмами

euph_d = pd.read_csv('/content/only_euphemism_dataset.csv')
euph_d = euph_d.dropna()

In [ ]:
len(euph_d)

In [ ]:
print(euph_d.isna().sum())

In [ ]:
# Присваиваем текстам метки 1 (если в тексте есть эвфемизм) или 0 (если нет) и удаляем теги <euphemism>
# Затем объединяем датасеты без эвфемизмов и проверенный датасет с эвфемизмами - получаем финальный датасет

In [ ]:
# Удаляет теги <euphemism> и </euphemism> из текста


def remove_euphemism_tags(text):
    if isinstance(text, str):  # Проверяем, является ли текст строкой
        return re.sub(r'<euphemism>|</euphemism>', '', text)
    return ""  # Возвращаем пустую строку для нестроковых значений

In [ ]:
# Добавляем метки
euph_d['label'] = 1  # Тексты с эвфемизмами
euph_d['utterance'] = euph_d['text'].apply(remove_euphemism_tags)
euph_d.drop('text', axis=1, inplace=True)
euph_d.drop('filename', axis=1, inplace=True)
euph_d.drop('index', axis=1, inplace=True)
euph_d.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
euph_d

In [ ]:
df_without_euphemism['label'] = 0  # Тексты без эвфемизмов
df_without_euphemism.drop('filename', axis=1, inplace=True)
df_without_euphemism.drop('index', axis=1, inplace=True)
df_without_euphemism.rename(columns={'text': 'utterance'}, inplace=True)

In [ ]:
df_without_euphemism

In [ ]:
# Объединяем датафреймы
combined_df = pd.concat([euph_d, df_without_euphemism], ignore_index=True)

# Вывод результата
print(combined_df)

In [ ]:
print(combined_df.isna().sum())

In [ ]:
len(combined_df)

In [ ]:
print(combined_df.head())

In [ ]:
combined_df.to_csv('dataset_full.csv')